In [82]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import regex as re

In [39]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [40]:
from selenium.webdriver.chrome.service import Service
s = Service(r"/usr/bin/chromedriver")


# Observations

In [135]:
def get_obs_fields(soup, obs_type = 'Field Observation', link = None):
    form_data = {}
    h1_tag = soup.find('h1', text=obs_type)
    if h1_tag:
        next_h2 = h1_tag.find_next_sibling('h2')
        if next_h2:
            location = next_h2.text.strip()
            form_data['locationName'] = location


    for label in soup.find_all('h6', {'class': 'nac-html-h6'}):
        key = label.text.strip()
        next_sibling = label.find_next_sibling()
        if next_sibling:
            form_data[key] = next_sibling.text.strip()


    kv_divs = soup.find_all('div', class_='nac-header-meta-bordered')
    for div in kv_divs:
        # Find the key
        key = div.find('h6').text.strip()
        value = None
        if div.find('h6').next_sibling is not None and not div.find('h6').next_sibling.name:
            value = div.find('h6').next_sibling.strip()
        else:
            # If value is not directly a sibling, it might be inside the next element.
            # We search for the first element inside the div but not the h6 itself.
            next_element = div.find('h6').find_next_sibling()
            if next_element:
                value = next_element.get_text(strip=True)

        cleaned_key = key.split(' <!-- ')[0]  # Splits the key and takes the first part, removing comments.

        form_data[cleaned_key] = value
        
    # Extracting Observation Summary
    obs_summary_div = soup.find('div', id='nac-obs-summary')
    if obs_summary_div:
        obs_summary = obs_summary_div.find('div', {'class': 'nac-tinymce nac-mb-3'})
        if obs_summary:
            form_data['Observation Summary'] = obs_summary.text.strip()

            
    h2_tag = soup.find(lambda tag: tag.name == "h2" and "Avalanche Summary" in tag.text)
    if h2_tag:
        summary_div = h2_tag.find_next_sibling()
        if summary_div:
            form_data['Avalanche Summary'] = summary_div.text 
        
    for instability in ['SIGNS OF INSTABILITY', "Instability Comments"]:
        instability_comments_div = soup.find('h4', string=instability)
        if instability_comments_div and instability_comments_div.parent:
            instability_comments = instability_comments_div.parent.find('div', {'class': 'nac-mb-3'})
            if instability_comments:
                form_data[instability] = instability_comments.text.strip()

    maps_link = soup.find('a', href=re.compile('^https://maps.google.com/maps'))
    if maps_link:
        gps_match = re.search(r'll=(-?\d+\.\d+),(-?\d+\.\d+)', maps_link['href'])
        if gps_match:
            lat, lng = gps_match.groups()
            form_data['Latitude'] = lat
            form_data['Longitude'] = lng
    snowflake_element = soup.find('i', class_='mdi mdi-snowflake')        
#    snowpack_header = soup.find('i', class_='mdi mdi-snowflake').find_parent('h2')
    snowpack_content = ''
    snowpack_photos = []
    if snowflake_element:
        try:
            snowpack_header = snowflake_element.find_parent('h2')
            for sibling in snowpack_header.find_next_siblings():
                if sibling.name == 'h2':
                    break
                snowpack_content += sibling.get_text(strip=True) + ' '
                # Extract photos
                figures = sibling.find_all('figure')
                for figure in figures:
                    photo_url = figure.find('div')['data-pswp-src']
                    snowpack_photos.append(photo_url)
            form_data['snowpack_description'] = snowpack_content
            form_data['snowpack_photos'] = ','.join(snowpack_photos)
        except:
            print('err snowpack')
    if link:
        form_data['link'] = link
    links = soup.find_all('a')

# Filter for the link that contains the pattern #/view/observations/
    observation_link = None
    for link in links:
        if 'href' in link.attrs and '#/view/observations/' in link['href']:
            observation_link = link['href']
            break
    form_data['observation_link'] = observation_link
    return form_data

    

In [78]:
driver = webdriver.Chrome(service=s)
driver.get(link)

# Wait for the table to load
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, ".nac-card.nac-observation-card.nac-card-hover"))
)

<selenium.webdriver.remote.webelement.WebElement (session="9a7835e321ceca4dd0c3a5e83e879f78", element="f.51366A8E0F48B2910672440166DE4267.d.2ECA1F5D92D32DF1C5CF8EA627FC1E77.e.42")>

In [190]:
pd.set_option('display.max_columns', None)
import pandas as pd
driver = webdriver.Chrome(service=s)
import time
for year in [2023,2024]:
    forms = []
    obs_df_export =pd.read_csv(f'../data/FAC_ob_export_{int(year-1)}-{int(year)}.csv')
    obs_df_export['link'] = obs_df_export['id'].apply(lambda x:f'https://flatheadavalanche.org/observations/read-observations/#/view/observations/{x}')
    for ind, (link, is_private) in enumerate(zip(obs_df_export.link,obs_df_export.private)):
        if ind%20==0:
            print(ind)
        if not is_private:
            driver.get(link)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".nac-card.nac-observation-card.nac-card-hover"))
            )
            time.sleep(3)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            forms.append(get_obs_fields(soup, link = link))
        time.sleep(2)
    obs_df = pd.DataFrame(forms)

    obs_df = obs_df[obs_df['Observation Date'].notnull()]
    obs_df['startDate'] = obs_df['Observation Date'].apply(lambda x:convert_to_iso_format(x))
    obs_df['startDate'].value_counts()#.head()
    obs_df = obs_df.rename(columns = {'Observer type':'observerType',
                                     'Activity':'activity','Zone / Region':'zone'})
    obs_df = obs_df[['zone',
                     'Cloud Cover','Temperature (F)',
           'New or Recent Snowfall (cm)', 'Rain/Snow Line',
           'Snow Available For Transport', 'Wind Loading', 'Weather Summary', 'observerType',
           'Observation Summary', 'SIGNS OF INSTABILITY', 'Instability Comments',
           'Route', 'Location', 'Distribution',
           'Sensitivity', 'Size', 'startDate','link','snowpack_description', 'snowpack_photos']]
    merge_cols = [col for col in obs_df.columns if col in obs_df_export.columns]
    print(len(obs_df_export))
    obs_df_export = obs_df_export.merge(obs_df,on=merge_cols,how='left')
    print(len(obs_df_export))
    obs_df_export.head()
    obs_df_export = obs_df_export.drop(columns = 'Location')#/view/observations/)
    BUCKET_NAME = 'avalanche-data'
    file_path = f'observations/general/fac_{int(year-1)}_{int(year)}.csv'
    print(file_path)
    upload_data_to_s3(obs_df_export, file_path)

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
573
573
observations/general/fac_2022_2023.csv
0
20
40
60
80
100
120
140
err snowpack
160
180
err snowpack
200
220
240
260
280
300
320
340
360
err snowpack
380
400
420
438
438
observations/general/fac_2023_2024.csv


In [284]:
import json
import copy
driver = webdriver.Chrome(service=s)
driver.get("https://www.snowpacktracker.com/fac/seasontracker")


In [41]:
driver = webdriver.Chrome(service=s)
driver.get(f"https://flatheadavalanche.org/observations/read-observations/#/view/observations?startDate=%22{int(year-1)}-10-01%22&endDate=%22{int(year)}-06-01%22&season={int(year)}")


In [292]:
import json
import copy

years = [2024]#:#2023,2024]
for year in years:
    driver = webdriver.Chrome(service=s)
    driver.get(f"https://flatheadavalanche.org/observations/read-observations/#/view/observations?startDate=%22{int(year-1)}-10-01%22&endDate=%22{int(year)}-06-01%22&season={int(year)}")

    # Wait for the table to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, ".nac-card.nac-observation-card.nac-card-hover"))
    )

    rows = driver.find_elements(By.CSS_SELECTOR, ".nac-card.nac-observation-card.nac-card-hover")
    rows[0].click()
    forms = []
    n=0
    while(True):
        driver.implicitly_wait(20)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        new_form = get_obs_fields(soup)
        if new_form in forms:
            break
        else:
            forms.append(copy.deepcopy(new_form))
        next_arrow = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, ".nac-btn.nac-btn-primary.nac-next"))
        )
        next_button = driver.find_element(By.CSS_SELECTOR, "a.nac-btn-primary.nac-next")
        next_button.click()
        time.sleep(20)
        n+=1
        print(len(forms))
        print(len(set([json.dumps(elem) for elem in forms])))
        print(new_form)
        print('\n')
    pd.DataFrame(forms).to_csv(f'/home/christine/Jupyter/snow/data/fac_obs_{int(year-1)}_{int(year)}.csv')

1
1
{'location': 'Rescue', 'Zone / Region': 'Flathead Range & Glacier NP', 'Observer Affiliation': 'Flathead Avalanche Center', 'Activity': 'Skiing/Snowboarding', 'Cloud Cover': 'Clear (No clouds)', 'Temperature (F)': '34 to 50', 'New or Recent Snowfall (cm)': 'None.', 'Rain/Snow Line': 'Not Observed', 'Snow Available For Transport': 'None', 'Wind Loading': 'None', 'Weather Summary': 'Clear skies and only a slight breeze in the afternoon.  Sunshine felt intense radiating off the snow.', 'Observation Date': 'Mar 17, 2024 (9:00) - Mar 17, 2024 (15:00)', 'Submission Date': 'Mar 17, 2024', 'Observer Name': 'Sarah Williams', 'Observer type': 'forecaster', 'Observation Summary': "The avalanches were calling, so we headed to Rescue Basin to document the recent avalanche activity after two nights without freezing temperatures.\n\nWe used ski crampons on the skin into the basin.\xa0 The surface crust remained supportable up to 4200 feet.\xa0\nTemperatures in the sun were hot! The snow surface w

9
9
{'location': 'Doris', 'Zone / Region': 'Swan Range', 'Observer Affiliation': 'Whitefish Professional Ski Patrol & The Patrol Fund', 'Activity': 'Snowmobiling/Snowbiking', 'Cloud Cover': 'Clear (No clouds)', 'Temperature (F)': '40', 'New or Recent Snowfall (cm)': 'Not Observed', 'Rain/Snow Line': 'Not Observed', 'Snow Available For Transport': 'Small Amounts', 'Wind Loading': 'None', 'Location': 'NESWNWNESESW', 'Distribution': 'IsolatedSpecificWidespread', 'Sensitivity': 'UnreactiveStubbornReactiveTouchy', 'Size': 'D1D2D3D4D5', 'Observation Date': 'Mar 15, 2024', 'Submission Date': 'Mar 15, 2024', 'Observer Name': 'JBurk', 'Observer type': 'professional', 'Observation Summary': 'Rollerballs on solar aspects, propagating fracture on an ECT', 'SIGNS OF INSTABILITY': 'Avalanches Observed', 'Instability Comments': 'Rollerballs were observed on solar aspects. on my ride out i noticed a large wet slide o the West aspect of Great Northern that looked to have run almost full path', 'Latitud

17
17
{'location': 'Bear Creek/wolf Creek', 'Zone / Region': 'Swan Range', 'Observer Affiliation': 'Patrol Fund educator', 'Activity': 'Skiing/Snowboarding', 'Route': 'Gained the ridge between bear creek and wolf creek. Started at valley floor and went up to 6,800 ft mostly on west and North facing terrain.After seeing no signs of instability, no results in our pits, and a lack of sun warming the slopes, we opened our standby terrain and skied 35-40 degree  plainer  slopes. We avoided any rocky terrain that might harbor or a shallower snowpack.', 'Cloud Cover': 'Broken (Mostly Cloudy, more than 4/8 but less than 8/8)', 'Temperature (F)': 'Not Observed', 'New or Recent Snowfall (cm)': '7cm', 'Rain/Snow Line': 'Not Observed', 'Snow Available For Transport': 'Moderate Amounts', 'Wind Loading': 'Previous', 'Weather Summary': 'Little to no wind', 'Observation Date': 'Mar 14, 2024', 'Submission Date': 'Mar 14, 2024', 'Observer type': 'professional', 'Observation Summary': 'We observed little

23
23
{'location': 'Dickey', 'Zone / Region': 'Flathead Range & Glacier NP', 'Activity': 'Skiing/Snowboarding', 'Observation Date': 'Mar 12, 2024', 'Submission Date': 'Mar 12, 2024', 'Observer type': 'public', 'Observation Summary': 'Rain turned to graupel around 4500Heavy 6-10” at 4600’, on the skin up. Ok the ski out anything below 5k wanted to slide or turn into roller balls. Intermittent gusty winds and intense graupel storms gave way to clearing skis and less wind. Lots of wind transport, one hollow collapse on open East facing terrain but couldn’t get any propagation in my pit. Recent avalanches on north face of elk and south facing chutes in north Dickey. Judging from debris looked like new/wet snow Submitted is an old crown for above Dickey lake, impressive width!', 'SIGNS OF INSTABILITY': 'Avalanches Observed  Collapsing Observed', 'Latitude': '48.402214', 'Longitude': '-114.13365'}


24
24
{'location': 'Lost Johnny', 'Zone / Region': 'Swan Range', 'Observer Affiliation': 'Fla

30
30
{'location': 'Spider bowl', 'Zone / Region': 'Other', 'Activity': 'Skiing/Snowboarding', 'Observation Date': 'Mar 9, 2024', 'Submission Date': 'Mar 9, 2024', 'Observer type': 'public', 'Observation Summary': "First things first, I observed some FAC-adjacent angels save my butt today on the road to the Strawberry Lake trailhead after burying my truck axel deep while trying to turn around. Be warned, trust the rut and think twice before trying to drive the road too far. There aren't any good places to turn around if you don't make to the trailhead.Thanks to those angels, however, we were able to ski today! We took the highway skinner to the Spider Bowl high point and skied a direct W aspect down to the creek, then maneuvered our way back to the vehicles. The snow got warm and moist at low elevations, but early afternoon high clouds and a brisk breeze seemed to keep the mid and upper elevation snow in good shape. We observed a few small roller balls on a due S aspect by some rocks, 

...

363
363
{'location': 'SE of Elk Mountain', 'Zone / Region': 'Flathead Range & Glacier NP', 'Observer Affiliation': 'Flathead Avalanche Center', 'Activity': 'Skiing/Snowboarding', 'Cloud Cover': 'Overcast (Cloudy, 8/8 covered)', 'Temperature (F)': '30', 'New or Recent Snowfall (cm)': '16 to 20', 'Rain/Snow Line': 'Not Observed', 'Snow Available For Transport': 'Small Amounts', 'Wind Loading': 'Previous', 'Weather Summary': 'Light to moderate winds throughout the day kept air temperature below freezing.', 'Observation Date': 'Dec 3, 2023 (9:00) - Dec 3, 2023 (16:00)', 'Submission Date': 'Dec 3, 2023', 'Observer Name': 'Sarah Williams', 'Observer type': 'forecaster', 'Observation Summary': 'We went to the eastern edge of the advisory area to check the snowpack..\n\nAt the trailhead, the 6 to 8 inches of recent snow barely covered the green bear grass.\nTrue to the east sides reputation, the wind blew light to moderate throughout the day\nWind scoured and wind drifts were the predominant s

371
371
{'location': 'McGee Creek', 'Zone / Region': 'Whitefish Range', 'Observer Affiliation': 'Flathead Nordic Backcountry Patrol', 'Activity': 'Skiing/Snowboarding', 'Route': "Apgar Crest to ~6400'. Mostly traveled on southerlies, but explored other aspects near and on the ridgetop.", 'Cloud Cover': 'Overcast (Cloudy, 8/8 covered)', 'Temperature (F)': 'Not Observed', 'New or Recent Snowfall (cm)': '1', 'Rain/Snow Line': 'Not Observed', 'Snow Available For Transport': 'Small Amounts', 'Wind Loading': 'None', 'Weather Summary': "The day began with high pressure and another inversion. The stratus deck hovered around 5000'. By mid afternoon the inversion began to dissipate and was replaced by overcast skies and light snow showers. Calm winds all afternoon.", 'Observation Date': 'Nov 30, 2023', 'Submission Date': 'Dec 1, 2023', 'Observer Name': 'Jackson George', 'Observer type': 'public', 'Observation Summary': "Climbed up to the Apgar Crest to assess conditions prior to the pattern shif

378
378
{'location': 'Whitefish Mountain', 'Zone / Region': 'Whitefish Range', 'Activity': 'Snowmobiling/Snowbiking', 'Observation Date': 'Nov 22, 2023', 'Submission Date': 'Nov 22, 2023', 'Observer Name': 'Kaleb Retz', 'Observer type': 'public', 'Observation Summary': 'Rode up to Whitefish Mountain into the upper bowls.Observed a Total Snow Depth of 95cm at 6900ft NE facing slope.-Facets near the ground- Pencil hard crust layer from 30 - 40 cm, was reactive on a shovel shear test- Surface rain crust at all elevations, we noticed the another crust 10cm from the surface at lower elevations', 'SIGNS OF INSTABILITY': 'None Reported', 'Latitude': '48.402214', 'Longitude': '-114.13365'}


379
379
{'location': 'Northern Flathead', 'Zone / Region': 'Flathead Range & Glacier NP', 'Activity': 'Skiing/Snowboarding', 'Observation Date': 'Nov 21, 2023', 'Submission Date': 'Nov 22, 2023', 'Observer type': 'public', 'Observation Summary': 'Found a wide array of snow conditions. Snow depth of 55-65 c

386
386
{'location': 'Western edge of the Flathead Range', 'Zone / Region': 'Flathead Range & Glacier NP', 'Observer Affiliation': 'Flathead Avalanche Center', 'Activity': 'Walking/Hiking', 'Route': 'Ousel Peak West trail', 'Cloud Cover': 'Obscured (Sky not visible due to fog or non-cloud layer)', 'Temperature (F)': 'Below freezing', 'New or Recent Snowfall (cm)': '30', 'Rain/Snow Line': '3500', 'Snow Available For Transport': 'Moderate Amounts', 'Wind Loading': 'None', 'Weather Summary': 'Obscured skies and cool temperatures above 5000 feet kept the snow surface cold and dry. Brief periods of afternoon sunshine below 4500 feet with snow falling from tree branches and sections of ground melting out.', 'Observation Date': 'Nov 16, 2023', 'Submission Date': 'Nov 16, 2023', 'Observer Name': 'Mark Dundas', 'Observer type': 'forecaster', 'Observation Summary': 'We traveled to the western edge of the Flathead Range to get a generalized overview of the snowpack.\xa0\n\nBefore the 11-15-23 sto

393
393
{'location': 'Late Middle Fork', 'Zone / Region': 'Flathead Range & Glacier NP', 'Activity': 'Walking/Hiking', 'Route': "Ascended NE ridge from 3800'-7300'", 'Cloud Cover': 'Overcast (Cloudy, 8/8 covered)', 'Temperature (F)': 'Below freezing', 'New or Recent Snowfall (cm)': 'Up to 25cm', 'Rain/Snow Line': 'Not Observed', 'Snow Available For Transport': 'Not Observed', 'Wind Loading': 'Not Observed', 'Observation Date': 'Nov 8, 2023', 'Submission Date': 'Nov 8, 2023', 'Observer type': 'public', 'Observation Summary': '5000\'-6500\':                      -Snow coverage became consistent at ~5000\' (1-2" of moist M/F grains(rain)) This rain crust existed to my high point of 7300\',  more on that                        below...                      -Powder snow around 5500\' with 4-8" of recent accumulation to 6500\' on top of the moist, M/F basal layer (4-6")6500\'-7300\':                      -Travel on foot became difficult above 6500\'.                       -Recent accumulatio

In [237]:
pd.DataFrame(forms).to_csv('/home/christine/Jupyter/snow/data/fac_observations_2023-11-01_2024-03-16.csv')

In [6]:
from utils import convert_to_iso_format

In [29]:
obs_df_export.head()
obs_df_export['link'] = obs_df_export['id'].apply(lambda x:f'https://flatheadavalanche.org/observations/read-observations/#/view/observations/{x}')#/view/observations/)

In [32]:
obs_df_export = obs_df_export.drop(columns = 'Location')

In [164]:
import importlib
import secrets
from secrets import *
importlib.reload(secrets)
import secrets
from secrets import SECRET_KEY,ACCESS_KEY
import boto3
import io
s3_client = boto3.client(
    's3',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY
)
BUCKET_NAME = 'avalanche-data'
file_path = 'observations/general/fac_2022_2023.csv'
upload_data_to_s3(obs_df_export, file_path)

def upload_data_to_s3(data, file_path):
    with io.StringIO() as csv_buffer:
        data.to_csv(csv_buffer, index=False)
        s3_client.put_object(Bucket=BUCKET_NAME, Key=file_path, Body=csv_buffer.getvalue())
def read_data_from_s3(file_path):
    csv_obj = s3_client.get_object(Bucket=BUCKET_NAME, Key=file_path)
    body = csv_obj['Body']
    csv_string = body.read().decode('utf-8')

    data = pd.read_csv(io.StringIO(csv_string))
    return data

# Avalanche

In [129]:
obs_df_export[obs_df_export.name=='Adam Clark']

,id,lastUpdated,startDate,status,private,locationName,locationPoint.lat,locationPoint.lng,obsSource,name,observerType,organization,activity,instability.cracking,instability.collapsing,instability.avalanches_caught,instability.avalanches_observed,instability.avalanches_triggered,instability.cracking_description,instability.collapsing_description,advancedFields.timeInField.end,advancedFields.timeInField.start,internalReporting,zone,internalReporting.partner.name,internalReporting.partner.type,link,Cloud Cover,Temperature (F),New or Recent Snowfall (cm),Rain/Snow Line,Snow Available For Transport,Wind Loading,Weather Summary,Observation Summary,SIGNS OF INSTABILITY,Instability Comments,Route,Distribution,Sensitivity,Size,snowpack_description,snowpack_photos
48,4ad2788b-7138-4405-bc78-5d77ea233770,2023-11-13T04:34:08.421944+00:00,2023-11-09,published,False,Glacier Park - East Side,48.4953,-113.4391,public,Adam Clark,public,NaN,"walking,climbing",False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://flatheadavalanche.org/observations/rea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,c9410c8e-da7e-4b60-81f2-48bd020a4e9f,2023-11-15T03:00:57.998571+00:00,2023-11-14,published,False,JFS Canyon - Running Rabbit Mtn,48.2357,-113.5368,public,Adam Clark,professional,BNSF Avalanche Safety,"driving,walking",False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,Flathead Range & Glacier NP,NaN,NaN,https://flatheadavalanche.org/observations/rea...,Clear (No clouds),Not Observed,0,Not Observed,Small Amounts,Not Observed,•\tMostly clear skies with no precipitation.•\...,•\tNo avalanche activity observed on the terra...,None Reported,NaN,•\tClimbed the Shed 11/10.7 ridge to the Shed ...,NaN,NaN,NaN,NaN,NaN
50,c8cc2d19-465f-4d28-b43f-4e18b8c354e2,2023-12-01T04:05:07.121159+00:00,2023-11-30,published,False,JFS Canyon - Snowslip Mtn,48.2534,-113.4943,public,Adam Clark,professional,BNSF Avalanche Safety,walking,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,Flathead Range & Glacier NP,NaN,NaN,https://flatheadavalanche.org/observations/rea...,"Broken (Mostly Cloudy, more than 4/8 but less ...",Not Observed,0,Not Observed,Small Amounts,Previous,•\tClear skies in JFS Canyon in the morning wi...,"•\t0-3” of snow on the Canyon floor, thicker p...",None Reported,•\tNo avalanche activity or signs of instabili...,•\tClimbed the Shed 6/7 ridge to the Shed 7 we...,NaN,NaN,NaN,NaN,NaN
54,2637d101-d307-44ac-920f-044b89adafb9,2023-11-19T19:57:50.792300+00:00,2023-11-17,published,False,Central Flathead Range,48.3456,-113.7939,public,Adam Clark,public,NaN,"skiing_snowboarding,walking,climbing",True,False,False,False,False,isolated,NaN,NaN,NaN,NaN,Flathead Range & Glacier NP,NaN,NaN,https://flatheadavalanche.org/observations/rea...,Clear (No clouds),Not Observed,Not Observed,Not Observed,Small Amounts,Previous,Clear skies with some valley fog. Light SW win...,"-Trace of crusty snow at 4200', 3-4"" of recent...",Cracking Observed (isolated),NaN,Ridge route to 8700' elevation. Some travel on...,NaN,NaN,NaN,NaN,NaN
72,fb38bb45-613e-4373-acce-c889beea98a7,2023-12-13T06:22:51.516524+00:00,2023-12-12,published,False,JFS Canyon - Running Rabbit Mtn,48.2485,-113.5369,public,Adam Clark,professional,BNSF Avalanche Safety,"skiing_snowboarding,walking",False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,Flathead Range & Glacier NP,NaN,NaN,https://flatheadavalanche.org/observations/rea...,"Overcast (Cloudy, 8/8 covered)",Not Observed,Not Observed,Not Observed,Small Amounts,Previous,•\tA sometimes-broken stratus deck of clouds h...,•\tNo avalanche activity observed.•\tNot much ...,None Reported,NaN,•\tClimbed the Shed 11/10.7 ridge to the Shed ...,NaN,NaN,NaN,NaN,NaN
85,f11c9adc-22d9-4ccd-8f14-d00de64bf379,2023-12-27T05:19:51.907576+00:00,2023-12-26,published,False,JFS Canyon - Running Rabbit Mtn,48.2498,-113.5215,widget,Adam Clark,professional,BNSF Avalanche Safety,"skiing_snowboarding,walking",False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,

In [127]:
av_df_export =pd.read_csv(f'../data/FAC_avy_export_{int(year-1)}-{int(year)}.csv')
av_df_export.head()
av_df_export['link'] = av_df_export['id'].apply(lambda x:f'https://flatheadavalanche.org/observations/read-observations/#/view/avalanches/{x}')
driver = webdriver.Chrome(service=s)
driver.get(av_df_export.iloc[0].link)
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, ".nac-card.nac-observation-card.nac-card-hover"))
)
time.sleep(3)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [133]:
associated_observation_container = soup.find(class_='nac-header-meta-bordered', string=lambda text: text and 'Associated Observation' in text)
associated_observation_container

In [140]:
def get_observation_link(soup):
    links = soup.find_all('a')
    observation_link = None
    for link in links:
        if 'href' in link.attrs and '#/view/observations/' in link['href']:
            observation_link = link['href']
            break
    return observation_link

In [152]:
pd.set_option('display.max_columns', None)
import pandas as pd
driver = webdriver.Chrome(service=s)
import time
for year in [2023]:
    dictos = []
    av_df_export =pd.read_csv(f'../data/FAC_avy_export_{int(year-1)}-{int(year)}.csv')
    av_df_export['link'] = av_df_export['id'].apply(lambda x:f'https://flatheadavalanche.org/observations/read-observations/#/view/avalanches/{x}')
    for ind, (link, is_private) in enumerate(zip(av_df_export.link,av_df_export.private)):
        if ind%20==0:
            print(ind)
        if not is_private:
            driver.get(link)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".nac-card.nac-observation-card.nac-card-hover"))
            )
            time.sleep(3)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            obs_link = get_observation_link(soup)
        else:
            print('private')
            obs_link = None
        dicto = {'link':link,'observation_link':obs_link}
        dictos.append(dicto)
        time.sleep(2)

0
20
40
60
80
100
120
140
160
180
200
220
240


In [ ]:
df = pd.DataFrame(dictos)
av_df_export = av_df_export.merge(df,on='link',how='left')

In [194]:
av_df_export['dSize'] = av_df_export['dSize'].fillna('Unk')
av_df_export['description'] = av_df_export.apply(lambda x:x['date'] + 
                                                 (' (approx)' if x['dateKnown'] == False else ' (exact)') + 
                                                 f'; dSize : D{x.dSize}; Number:{x.number}',axis=1)

av_df_export['description'] = av_df_export['description'].apply(lambda x:x.replace('DUnk','Unk'))
av_df_export['avalanche_link_text'] = av_df_export.apply(lambda x: f'<a href="{x.link}" target="_blank">{x.description}</a>',axis=1)
grp = av_df_export.groupby('observation_id').agg({'avalanche_link_text':lambda x:' '.join(x)}).reset_index().rename(columns = {'observation_id':'id'})
observations = read_data_from_s3(file_path = f'observations/general/fac_{int(year-1)}_{int(year)}.csv')
observations = observations.merge(grp,on='id',how='left')
observations

,id,lastUpdated,startDate,status,private,locationName,locationPoint.lat,locationPoint.lng,obsSource,name,observerType,organization,activity,instability.cracking,instability.collapsing,instability.avalanches_caught,instability.avalanches_observed,instability.avalanches_triggered,instability.cracking_description,instability.collapsing_description,advancedFields.timeInField.end,advancedFields.timeInField.start,internalReporting,zone,internalReporting.partner.name,internalReporting.partner.type,link,Cloud Cover,Temperature (F),New or Recent Snowfall (cm),Rain/Snow Line,Snow Available For Transport,Wind Loading,Weather Summary,Observation Summary,SIGNS OF INSTABILITY,Instability Comments,Route,Distribution,Sensitivity,Size,snowpack_description,snowpack_photos,avalanche_link_text
0,64303d6e-7a59-422b-a49d-a56d6972a504,2023-01-23T19:39:55.567281+00:00,2023-01-22,published,True,Lone Pine,48.5060,-114.3460,dashboard,blase reardon for WMR Patrol,forecaster,FAC,other,False,False,False,True,True,NaN,NaN,NaN,NaN,NaN,Whitefish Range,NaN,NaN,https://flatheadavalanche.org/observations/rea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<a href=""https://flatheadavalanche.org/observa..."
1,77908a50-7890-40a3-8b21-e120d489229b,2023-01-04T00:13:19.878422+00:00,2023-01-03,published,False,Running Rabbit,48.2589,-113.5409,public,Zach Rutt,public,NaN,skiing_snowboarding,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,Flathead Range & Glacier NP,NaN,NaN,https://flatheadavalanche.org/observations/rea...,Obscured (Sky not visible due to fog or non-cl...,Not Observed,Not Observed,Not Observed,Not Observed,Not Observed,Visibility was limited,We did not observe any signs of instability.,None Reported,NaN,"Snow Slip to Running Rabbit, out Goat Lick Path.",NaN,NaN,NaN,We found the Santa soaker crust present at our...,NaN,NaN
2,f1943eb1-5d02-4478-82ae-76ab519938a5,2023-01-04T03:44:15.497449+00:00,2023-01-03,published,False,Mid Swan,47.5240,-113.7020,dashboard,Rob Millspaugh,forecaster,FAC,"skiing_snowboarding,snowmobiling_snowbiking",False,False,False,False,False,NaN,NaN,16.0,11.0,NaN,NaN,NaN,NaN,https://flatheadavalanche.org/observations/rea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,e2bfc2bf-dee0-4322-935a-02cd4ce08182,2023-01-04T03:59:40.969674+00:00,2023-01-03,published,False,Middle Fork Corridor,48.4025,-113.7886,public,Sara Kovalsky,public,NaN,skiing_snowboarding,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,Flathead Range & Glacier NP,NaN,NaN,https://flatheadavalanche.org/observations/rea...,"Overcast (Cloudy, 8/8 covered)",14,Not Observed,Not Observed,Small Amounts,NaN,Cloudy with on and off light flurries througho...,Our group travelled on west through northeast ...,None Reported,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2c197f2a-26dd-47e7-a752-bd42bc209e01,2023-01-13T15:43:28.803894+00:00,2023-01-12,published,False,Snowslip,48.2560,-113.4920,dashboard,Rob Millspaugh,forecaster,FAC,skiing_snowboarding,False,False,False,False,False,NaN,NaN,13.5,9.5,NaN,Flathead Range & Glacier NP,NaN,NaN,https://flatheadavalanche.org/observations/rea...,"Broken (Mostly Cloudy, more than 4/8 but less ...",28,1 inch,Not Observed,Small Amounts,Light,S-1 started at 1100,Teamed up with USGS on a visit to Snowslip:\n-...,None Reported,No obvious signs of instability,Ascend southerly aspect to ridge. Traverse ri...,NaN,NaN,NaN,"- ECTX in pit @ 6500 feet, SE aspect (see atta...",https://avalanche-org-media.s3.us-west-2.amazo...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,2b4b7c13-578c-4b3f-af45-78df1e15fc8f,2023-04-20T01:43:24.770929+00:00,2023-04-19,published,False,Tranquil Basin,48.2236,-113.4992,public,David Kerner,public,NaN,skiing_snowboarding,True,False,False,True,False,isolated,NaN,NaN,NaN,NaN,Flathead Range & Glacier NP,NaN,NaN,https://flatheadavalanche.org/observations/rea...,"

In [196]:
year = 2023
upload_data_to_s3(observations,  file_path = f'observations/general/fac_{int(year-1)}_{int(year)}.csv')

In [ ]:
av_df_export[~av_df_export.observation_link.notnull()]
mask = av_df_export.observation_link.notnull()
av_df_export.loc[mask,'observation_id'] = av_df_export.loc[mask,'observation_link'].apply(lambda x:x.split('/')[-1])
av_df_export.head()

In [189]:
f'observations/general/fac_{int(year-1)}_{int(year)}.csv'

'observations/general/fac_2022_2023.csv'

In [188]:
observations = read_data_from_s3(file_path = f'observations/general/fac_{int(year-1)}_{int(year)}.csv')
observations

,id,lastUpdated,startDate,status,private,locationName,locationPoint.lat,locationPoint.lng,obsSource,name,observerType,organization,activity,instability.cracking,instability.collapsing,instability.avalanches_caught,instability.avalanches_observed,instability.avalanches_triggered,instability.cracking_description,instability.collapsing_description,advancedFields.timeInField.end,advancedFields.timeInField.start,internalReporting,zone,internalReporting.partner.name,internalReporting.partner.type,link,Cloud Cover,Temperature (F),New or Recent Snowfall (cm),Rain/Snow Line,Snow Available For Transport,Wind Loading,Weather Summary,Observation Summary,SIGNS OF INSTABILITY,Instability Comments,Route,Distribution,Sensitivity,Size,snowpack_description,snowpack_photos
0,0a442151-b15d-4f96-b3ec-fa2042f65d31,2023-12-26T00:58:36.450904+00:00,2023-12-25,published,False,East of WMR,48.4991,-114.3149,widget,Otto Hansen,public,NaN,skiing_snowboarding,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,Whitefish Range,NaN,NaN,https://flatheadavalanche.org/observations/rea...,"Few (Mostly Clear, up to 2/8 covered)",Not Observed,Not Observed,Not Observed,Small Amounts,NaN,NaN,Small weak layer sitting around 40cm deep. La...,None Reported,NaN,"Traversed across the ridge, dug a pit around 6...",NaN,NaN,NaN,Significant surface hoar on all aspects. Larg...,NaN
1,ba3e5b6d-c7fd-4b0e-a83b-6a87a6559ad5,2024-02-04T05:11:47.766058+00:00,2023-12-14,published,False,Central Glacier Park,48.6320,-113.8230,dashboard,Sarah Williams,forecaster,Flathead Avalanche Center,"walking,xcskiing_snowshoeing",False,False,False,False,False,NaN,NaN,17.5,7.5,NaN,Flathead Range & Glacier NP,Mark Dundas,forecaster,https://flatheadavalanche.org/observations/rea...,"Overcast (Cloudy, 8/8 covered)",Below freezing,0,Not Observed,Small Amounts,NaN,Calm day with a two layer inversion. Low eleva...,We got an early start and headed to Glacier Pa...,None Reported,No recent avalanches or signs of instability o...,"We traveled mostly on a trail on southwest, we...",NaN,NaN,NaN,We had no propagating results in our large col...,https://avalanche-org-media.s3.us-west-2.amazo...
2,b2367b15-195f-4c7b-a2eb-e87a8ab1c507,2023-12-15T03:36:29.366651+00:00,2023-12-14,published,False,Crown Bowl,48.1548,-113.9345,public,Paul Fotter,public,NaN,skiing_snowboarding,False,False,True,True,True,NaN,NaN,NaN,NaN,NaN,Swan Range,NaN,NaN,https://flatheadavalanche.org/observations/rea...,"Broken (Mostly Cloudy, more than 4/8 but less ...",Not Observed,0,Not Observed,Not Observed,NaN,Intermittent sun early with clouds developing ...,Intermittent sunshine and calm winds accompani...,Avalanches Observed Avalanches Triggered Cau...,Throughout the tour we checked for weak layers...,We toured from Camp Misery to the top of Radio...,NaN,NaN,NaN,We found the 12/7 rain crust up to our high po...,NaN
3,cc3b8680-a531-40b6-b5e0-1c8e91cb4f0e,2024-01-06T23:21:49.175261+00:00,2024-01-06,published,False,Peter's Ridge,48.2065,-114.0220,widget,Josh McElrath,public,NaN,skiing_snowboarding,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,Swan Range,NaN,NaN,https://flatheadavalanche.org/observations/rea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Testing the waters here to see the coverage et...,None Reported,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,db5e09d6-1149-4e7a-b33c-347bd1f1516d,2023-12-25T13:23:39.100617+00:00,2023-12-24,published,False,North forty,48.9200,-114.8700,dashboard,blase reardon,forecaster,Flathead Avalanche Center,skiing_snowboarding,False,False,False,True,True,NaN,NaN,16.0,10.0,NaN,Whitefish Range,NaN,NaN,https://flatheadavalanche.org/observations/rea...,Clear (No clouds),teens,5-20,Not Observed,Small Amounts,Previous,Clear skies in AM with cold temps (19* at car)...,"It's not only not all bad, it's very good in p...",Avalanches Observed Avalanches Triggered,No cracking or collapsing observed.,NaN,IsolatedSpecificWidespread,UnreactiveStubbornReactiveTouchy,D1D2D3D4D5,Warming midweek left 3-8 cm thick Melt-Freeze ...,NaN
...,...,...,...,...,.

In [150]:
observations = 

In [147]:
BUCKET_NAME = 'avalanche-data'
file_path = f'observations/avalanche/fac_{int(year-1)}_{int(year)}.csv'
upload_data_to_s3(av_df_export, file_path)

In [137]:
av_df = pd.DataFrame(forms)
av_df

,locationName,Occurrence Time,Zone / Region,Observer Affiliation,Associated Observation,# Avalanches:,Aspect:,Elevation:,Vertical Fall:,Destructive Size:,Relative Size:,Type:,Slab Thickness:,Width:,Trigger:,Occurrence Date,Observer type,Latitude,Longitude,link,observation_link,Avalanche Summary
0,Cascadilla,Not Reported,Flathead Range & Glacier NP,fac,View Observation,1,NE,6600 ft,500 ft,D3,R3,HS-Hard Slab,100 cm (3.5 ft),600 ft,N-Natural,"Feb 20, 2023 (Estimated)",forecaster,48.402214,-114.13365,https://flatheadavalanche.org/observations/rea...,#/view/observations/b2a468df-3329-4760-a3ab-15...,NaN
1,test,Not Reported,Flathead Range & Glacier NP,fac,View Observation,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Dec 1, 2022 (Exact)",public,48.402214,-114.13365,https://flatheadavalanche.org/observations/rea...,#/view/observations/f3445fdb-bb99-41c3-970c-2c...,test
2,Red Meadow,Not Reported,Whitefish Range,fac,View Observation,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Dec 25, 2022 (Estimated)",public,48.402214,-114.13365,https://flatheadavalanche.org/observations/rea...,#/view/observations/e0fd70d2-b9d7-4980-8f40-48...,NaN
3,Cascadilla,Not Reported,Flathead Range & Glacier NP,fac,View Observation,1,NE,5200 ft,NaN,D1,NaN,SS-Soft Slab,12 cm (0.5 ft),NaN,AS-Skierc-Intentional,"Jan 22, 2023 (Exact)",public,48.402214,-114.13365,https://flatheadavalanche.org/observations/rea...,#/view/observations/b6046698-d3ed-47e7-a045-93...,Thin wind slab
4,Red Meadow,19:22 (Estimated),Whitefish Range,fac,View Observation,3,NaN,NaN,NaN,D3,NaN,NaN,NaN,NaN,NaN,"Dec 26, 2022 (Estimated)",forecaster,48.402214,-114.13365,https://flatheadavalanche.org/observations/rea...,#/view/observations/a98470af-0ec9-4302-85fc-eb...,NaN
5,N. Whitefish Range,13:31 (Exact),Whitefish Range,fac,View Observation,3,NaN,NaN,NaN,D1,NaN,NaN,NaN,NaN,AS-Skierc-Intentional,"Jan 17, 2023 (Exact)",public,48.402214,-114.13365,https://flatheadavalanche.org/observations/rea...,#/view/observations/79bbe6f9-a4d0-4c3c-a4c8-ae...,NaN


In [139]:
av_df_export.date

0      2023-02-20
1      2022-12-01
2      2022-12-25
3      2023-01-22
4      2022-12-26
          ...    
250    2023-04-02
251    2023-04-01
252    2023-04-02
253    2023-04-23
254    2023-04-23
Name: date, Length: 255, dtype: object

In [ ]:
    av_df = pd.DataFrame(forms)

    av_df = obs_df[obs_df['Observation Date'].notnull()]
    obs_df['startDate'] = obs_df['Observation Date'].apply(lambda x:convert_to_iso_format(x))
    obs_df['startDate'].value_counts()#.head()
    obs_df = obs_df.rename(columns = {'Observer type':'observerType',
                                     'Activity':'activity','Zone / Region':'zone'})
    obs_df = obs_df[['zone',
                     'Cloud Cover','Temperature (F)',
           'New or Recent Snowfall (cm)', 'Rain/Snow Line',
           'Snow Available For Transport', 'Wind Loading', 'Weather Summary', 'observerType',
           'Observation Summary', 'SIGNS OF INSTABILITY', 'Instability Comments',
           'Route', 'Location', 'Distribution',
           'Sensitivity', 'Size', 'startDate','link','snowpack_description', 'snowpack_photos']]
    merge_cols = [col for col in obs_df.columns if col in obs_df_export.columns]
    print(len(obs_df_export))
    obs_df_export = obs_df_export.merge(obs_df,on=merge_cols,how='left')
    print(len(obs_df_export))
    obs_df_export.head()
    obs_df_export = obs_df_export.drop(columns = 'Location')#/view/observations/)
    BUCKET_NAME = 'avalanche-data'
    file_path = f'observations/avalanche/fac_{int(year-1)}_{int(year)}.csv'
    upload_data_to_s3(obs_df_export, file_path)

In [274]:
import json
import copy
driver = webdriver.Chrome(service=s)
season = 2023
driver.get(f"https://flatheadavalanche.org/observations/read-observations/#/view/avalanches?startDate=%22{int(season-1)}-11-01%22&endDate=%22{int(season)}-06-16%22&season={int(season)}")

# Wait for the table to load
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, ".nac-card.nac-observation-card.nac-card-hover"))
)

rows = driver.find_elements(By.CSS_SELECTOR, ".nac-card.nac-observation-card.nac-card-hover")
rows[0].click()
forms = []
n=0
while(True):
    driver.implicitly_wait(20)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    new_form = get_obs_fields(soup, obs_type = 'Avalanche Occurrence')
    if new_form in forms:
        break
    else:
        forms.append(copy.deepcopy(new_form))
    next_arrow = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, ".nac-btn.nac-btn-primary.nac-next"))
    )
    next_button = driver.find_element(By.CSS_SELECTOR, "a.nac-btn-primary.nac-next")
    next_button.click()
    time.sleep(20)
    n+=1
    print(len(forms))
    print(len(set([json.dumps(elem) for elem in forms])))
    print(new_form)
    print('\n')
pd.DataFrame(forms).to_csv('/home/christine/Jupyter/snow/data/fac_avalanche_2022_2023.csv')

1
1
{'Latitude': '48.402214', 'Longitude': '-114.13365'}


2
2
{'location': 'Rescue Creek - south chutes', 'Occurrence Time': 'Not Reported', 'Zone / Region': 'Flathead Range & Glacier NP', 'Observer Affiliation': 'fac', 'Associated Observation': 'View Observation', '# Avalanches:': '12', 'Aspect:': 'SE', 'Slope Angle:': '35°', 'Elevation:': '6000 ft', 'Vertical Fall:': '1000 ft', 'Destructive Size:': 'D1.5', 'Relative Size:': 'R2', 'Type:': 'WL-Wet Loose', 'Bed Surface:': 'I-New/Old Interface', 'Width:': '50 ft', 'Trigger:': 'N-Natural', 'Occurrence Date': 'Apr 23, 2023 (Exact)', 'Observer type': 'forecaster', 'Avalanche Summary': 'Sustained sun and warming intitiated these slides. ', 'Latitude': '48.402214', 'Longitude': '-114.13365'}


3
3
{'location': 'Rescue Creek - south chutes', 'Occurrence Time': 'Not Reported', 'Zone / Region': 'Flathead Range & Glacier NP', 'Observer Affiliation': 'fac', 'Associated Observation': 'View Observation', '# Avalanches:': '3', 'Aspect:': 'SE', 'Slo

15
15
{'location': 'Upper Wahoo', 'Zone / Region': 'Flathead Range & Glacier NP', 'Observer Affiliation': 'fac', 'Associated Observation': 'View Observation', '# Avalanches:': '1', 'Aspect:': 'NE', 'Slope Angle:': '35°', 'Elevation:': '6200 ft', 'Vertical Fall:': '200 ft', 'Destructive Size:': 'D1.5', 'Relative Size:': 'R3', 'Type:': 'SS-Soft Slab', 'Bed Surface:': 'I-New/Old Interface', 'Slab Thickness:': '30 cm (1 ft)', 'Width:': '50 ft', 'Trigger:': 'AR-Snowboarderu-Unintentional', 'Occurrence Date': 'Apr 2, 2023 (Exact)', 'Occurrence Time': '00:20 (Exact)', 'Observer type': 'public', 'Avalanche Summary': 'This avalanche was unintentionally triggered at the flank, from a connected lower angle slope while descending. The slab propagated up (well above the rider) and across the small slope. Rider was easily able to avoid getting caught. ', 'Latitude': '48.402214', 'Longitude': '-114.13365'}


16
16
{'location': 'Flathead', 'Zone / Region': 'Flathead Range & Glacier NP', 'Observer Affi

28
28
{'location': 'Swan Lake', 'Zone / Region': 'Swan Range', 'Observer Affiliation': 'fac', 'Associated Observation': 'View Observation', '# Avalanches:': '1', 'Aspect:': 'NW', 'Slope Angle:': '35°', 'Elevation:': '7100 ft', 'Vertical Fall:': '100 ft', 'Destructive Size:': 'D1', 'Relative Size:': 'R1', 'Type:': 'SS-Soft Slab', 'Bed Surface:': 'S-New Snow', 'Slab Thickness:': '10 cm (0.5 ft)', 'Width:': '40 ft', 'Trigger:': 'AS-Skiery-Sympathetic', 'Occurrence Date': 'Mar 15, 2023 (Exact)', 'Occurrence Time': '13:45 (Exact)', 'Observer type': 'forecaster', 'Avalanche Summary': 'Avalanche failed on adjacent slope after intentionally triggering a small avalanche. ', 'Latitude': '48.402214', 'Longitude': '-114.13365'}


29
29
{'location': 'Swan Lake', 'Zone / Region': 'Swan Range', 'Observer Affiliation': 'fac', 'Associated Observation': 'View Observation', '# Avalanches:': '1', 'Aspect:': 'N', 'Slope Angle:': '35°', 'Elevation:': '7200 ft', 'Vertical Fall:': '200 ft', 'Destructive Size:

42
42
{'location': 'Green Mountain', 'Occurrence Time': 'Not Reported', 'Zone / Region': 'Whitefish Range', 'Observer Affiliation': 'fac', 'Associated Observation': 'View Observation', '# Avalanches:': '1', 'Aspect:': 'E', 'Slope Angle:': '38°', 'Elevation:': '7600 ft', 'Vertical Fall:': '800 ft', 'Destructive Size:': 'D2', 'Relative Size:': 'R2', 'Type:': 'HS-Hard Slab', 'Bed Surface:': 'O-Old Snow', 'Slab Thickness:': '60 cm (2 ft)', 'Width:': '200 ft', 'Trigger:': 'AM-Snowmobileu-Unintentional', 'Occurrence Date': 'Mar 7, 2023 (Estimated)', 'Observer type': 'forecaster', 'Avalanche Summary': 'Original avalanche observation is posted on 3/8/23. ', 'Latitude': '48.402214', 'Longitude': '-114.13365'}


43
43
{'location': 'Nyack Mountain, northeast face', 'Occurrence Time': 'Not Reported', 'Zone / Region': 'Flathead Range & Glacier NP', 'Observer Affiliation': 'fac', 'Associated Observation': 'View Observation', '# Avalanches:': '1', 'Aspect:': 'NE', 'Elevation:': '7000 ft', 'Destructiv

54
54
{'location': 'Elk Moutain - Middle Path', 'Occurrence Time': 'Not Reported', 'Zone / Region': 'Other', 'Observer Affiliation': 'fac', 'Associated Observation': 'View Observation', '# Avalanches:': '1', 'Aspect:': 'E', 'Slope Angle:': '40°', 'Elevation:': '6700 ft', 'Vertical Fall:': '1000 ft', 'Destructive Size:': 'D2.5', 'Relative Size:': 'R2', 'Type:': 'HS-Hard Slab', 'Bed Surface:': 'U-Unknown', 'Slab Thickness:': '100 cm (3.5 ft)', 'Width:': '250 ft', 'Trigger:': 'N-Natural', 'Occurrence Date': 'Feb 28, 2023 (Estimated)', 'Observer type': 'forecaster', 'Latitude': '48.402214', 'Longitude': '-114.13365'}


55
55
{'location': 'Flattop Mountain', 'Occurrence Time': 'Not Reported', 'Zone / Region': 'Other', 'Observer Affiliation': 'fac', 'Associated Observation': 'View Observation', '# Avalanches:': '1', 'Aspect:': 'N', 'Elevation:': '6500 ft', 'Destructive Size:': 'D2', 'Type:': 'HS-Hard Slab', 'Bed Surface:': 'U-Unknown', 'Slab Thickness:': '60 cm (2 ft)', 'Trigger:': 'NC-Corni

...

189
189
{'location': 'Cascadilla N Side', 'Zone / Region': 'Flathead Range & Glacier NP', 'Observer Affiliation': 'fac', 'Associated Observation': 'View Observation', '# Avalanches:': '1', 'Aspect:': 'N', 'Slope Angle:': '40°', 'Elevation:': '5500 ft', 'Vertical Fall:': '1000 ft', 'Destructive Size:': 'D2', 'Relative Size:': 'R3.5', 'Type:': 'HS-Hard Slab', 'Bed Surface:': 'O-Old Snow', 'Slab Thickness:': '40 cm (1.5 ft)', 'Width:': '200 ft', 'Trigger:': 'N-Natural', 'Occurrence Date': 'Dec 26, 2022 (Estimated)', 'Occurrence Time': '20:27 (Estimated)', 'Observer type': 'forecaster', 'Avalanche Summary': 'Upper crown appeared to be a dry slab. Lower crown looked more like it failed as a wet slab given the jagged nature of the crown line. ', 'Latitude': '48.402214', 'Longitude': '-114.13365'}


190
190
{'location': 'Red Meadow', 'Occurrence Time': 'Not Reported', 'Zone / Region': 'Whitefish Range', 'Observer Affiliation': 'fac', 'Associated Observation': 'View Observation', '# Avalanches

202
202
{'location': 'SE face of Running Rabbit Mtn - Railroad Shed 10 feeder path', 'Occurrence Time': 'Not Reported', 'Zone / Region': 'Flathead Range & Glacier NP', 'Observer Affiliation': 'fac', 'Associated Observation': 'View Observation', '# Avalanches:': '1', 'Aspect:': 'SE', 'Destructive Size:': 'D2', 'Type:': 'SS-Soft Slab', 'Bed Surface:': 'O-Old Snow', 'Slab Thickness:': '100 cm (3.5 ft)', 'Trigger:': 'N-Natural', 'Occurrence Date': 'Dec 20, 2022 (Exact)', 'Observer type': 'public', 'Avalanche Summary': "Runout was at least to 5400', likely further, but out of view. Likely this failed on Nov. facets given the slab thickness.", 'Latitude': '48.402214', 'Longitude': '-114.13365'}


203
203
{'location': 'South face of Running Rabbit Mtn - Railroad Path 1163', 'Occurrence Time': 'Not Reported', 'Zone / Region': 'Flathead Range & Glacier NP', 'Observer Affiliation': 'fac', 'Associated Observation': 'View Observation', '# Avalanches:': '1', 'Aspect:': 'S', 'Elevation:': '7100 ft',

215
215
{'location': 'Shed 7 Ridge (BNSF test slopes)', 'Occurrence Time': 'Not Reported', 'Zone / Region': 'Flathead Range & Glacier NP', 'Observer Affiliation': 'fac', 'Associated Observation': 'View Observation', '# Avalanches:': '2', 'Aspect:': 'NE', 'Elevation:': '6100 ft', 'Vertical Fall:': '50 ft', 'Destructive Size:': 'D1', 'Relative Size:': 'R2', 'Type:': 'SS-Soft Slab', 'Bed Surface:': 'I-New/Old Interface', 'Slab Thickness:': '30 cm (1 ft)', 'Width:': '30 ft', 'Trigger:': 'AS-Skierc-Intentional', 'Occurrence Date': 'Dec 9, 2022 (Exact)', 'Observer type': 'public', 'Latitude': '48.402214', 'Longitude': '-114.13365'}


216
216
{'location': 'Slopes above the Beaver pond', 'Occurrence Time': 'Not Reported', 'Zone / Region': 'Whitefish Range', 'Observer Affiliation': 'fac', 'Associated Observation': 'View Observation', '# Avalanches:': '1', 'Aspect:': 'E', 'Slope Angle:': '40°', 'Elevation:': '6500 ft', 'Vertical Fall:': '100 ft', 'Destructive Size:': 'D2', 'Relative Size:': 'R2.

229
229
{'location': 'test', 'Occurrence Time': 'Not Reported', 'Zone / Region': 'Flathead Range & Glacier NP', 'Observer Affiliation': 'fac', 'Associated Observation': 'View Observation', '# Avalanches:': '1', 'Occurrence Date': 'Dec 1, 2022 (Exact)', 'Observer type': 'public', 'Avalanche Summary': 'test', 'Latitude': '48.402214', 'Longitude': '-114.13365'}


230
230
{'location': '48.0N -113.0W', 'Occurrence Time': 'Not Reported', 'Zone / Region': 'Swan Range', 'Observer Affiliation': 'fac', 'Associated Observation': 'View Observation', '# Avalanches:': '1', 'Aspect:': 'S', 'Elevation:': '5000 ft', 'Vertical Fall:': '500 ft', 'Destructive Size:': 'D1', 'Relative Size:': 'R1', 'Type:': 'SS-Soft Slab', 'Bed Surface:': 'I-New/Old Interface', 'Slab Thickness:': '100 cm (3.5 ft)', 'Width:': '50 ft', 'Trigger:': 'U-Unknownc-Intentional', 'Occurrence Date': 'Dec 1, 2022 (Exact)', 'Observer type': 'public', 'Avalanche Summary': 'text text text  more text text', 'Latitude': '48.402214', 'Longi

243
243
{'location': 'Jewel Basin', 'Zone / Region': 'Swan Range', 'Observer Affiliation': 'fac', 'Associated Observation': 'View Observation', '# Avalanches:': '1', 'Aspect:': 'NE', 'Slope Angle:': '45°', 'Elevation:': '6000 ft', 'Vertical Fall:': '10 ft', 'Destructive Size:': 'D1', 'Type:': 'SS-Soft Slab', 'Bed Surface:': 'O-Old Snow', 'Slab Thickness:': '20 cm (0.5 ft)', 'Width:': '30 ft', 'Trigger:': 'N-Naturalr-Remote', 'Occurrence Date': 'Nov 23, 2022 (Exact)', 'Occurrence Time': '16:15 (Exact)', 'Observer type': 'forecaster', 'Avalanche Summary': 'Small slab on the lee side of a short, steep rollover.', 'Latitude': '48.402214', 'Longitude': '-114.13365'}


244
244
{'location': 'test firefox', 'Occurrence Time': 'Not Reported', 'Zone / Region': 'Swan Range', 'Observer Affiliation': 'fac', 'Associated Observation': 'View Observation', '# Avalanches:': '1', 'Aspect:': 'SW', 'Elevation:': '3000 ft', 'Destructive Size:': 'D2.5', 'Type:': 'U-Unknown', 'Slab Thickness:': '100 cm (3.5 f

In [276]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)